

-1 represents value not set

(value of X, value of Y, new probability of X, new probability of Y)

Note, there will only be a new probability if the value was not set.

(p1, p2) 

In [14]:
import random
import numpy as np
import math
from constraint import ProbMinExposed
import networkx as nx
import matplotlib.pyplot as plt
from collections import defaultdict
from ortools.linear_solver import pywraplp
from ortools.linear_solver.pywraplp import Constraint, Solver, Variable, Objective

from typing import Set, Dict
import pickle as pkl


In [10]:
data_name = "mon10k"
data_dir = f"../data/mont/labelled/{data_name}"
G: nx.Graph = nx.read_edgelist(f"{data_dir}/data.txt", nodetype=int)

In [17]:
# Set of initial infected
N = G.number_of_nodes()
NUM_INFECTED = int(N / 20)

rand_infected = np.random.choice(N, NUM_INFECTED, replace=False)
I_SET = set(rand_infected)
#print(f"Infected: {I_SET}")

# COSTS = np.random.randint(1, 20, size=N)
COSTS = np.ones(N)
print(f"COSTS: {COSTS}")
# Compute distances
dist_dict = nx.multi_source_dijkstra_path_length(G, I_SET)

# convert dict vertex -> distance
# to distance -> [vertex]
level_dists = defaultdict(set)
for (i, v) in dist_dict.items():
    level_dists[v].add(i)

# print(level_dists)

I: Set[int] = level_dists[0]

# Obtain V_1 and V_2

# Set of vertices distance 1 away from infected I
V1: Set[int] = level_dists[1]

# Set of vertices distance 2 away from infected I
V2: Set[int] = level_dists[2]

COSTS: [1. 1. 1. ... 1. 1. 1.]


In [18]:
c = ProbMinExposed(G,I,V1,V2,10,COSTS)

TypeError: __init__() missing 4 required positional arguments: 'p1', 'q', 'k', and 'costs'

In [31]:
def iterated_round(problem: ProbMinExposed, d: int):
    problem.solve_lp()
    probabilities = np.array(problem.getVariables())
    
    curr = 0
    
    while curr + d < len(probabilities):
        
        probabilities[curr:curr+d] = D_prime(probabilities[curr:curr+d])
        
        for i in range(d):
            
            problem.setVariable(curr+i, probabilities[curr+i])
        
        problem.solve_lp()
        probabilities = np.array(problem.getVariables())
        
        curr += d
    
    probabilities[curr:] = D_prime(probabilities[curr:])
    
    return probabilities

In [29]:
def basic_integer_round(problem: ProbMinExposed):
    problem.solve_lp()
    probabilities = problem.getVariables()
    return D(np.array(probabilities))

In [19]:
def simplify(alpha:float, beta:float):
    if (alpha<0) | (alpha>1) | (beta<0) | (beta>1):
        raise ValueError('Invalid alpha or beta')
    
    p = random.uniform(0,1)
    
    if alpha+beta==0:
        
        return (0,0,-1,-1)
    
    elif alpha+beta<1:
        
        if p<alpha/(alpha+beta):
            return (-1,0,alpha+beta,-1)
        else:
            return (0,-1,-1,alpha+beta)
        
    elif alpha+beta==1:
        
        if p<alpha:
            return (1,0,-1,-1)
        else:
            return (0,1,-1,-1)
        
    elif alpha+beta < 2:
        
        if p<(1-beta)/(2-alpha-beta):
            return (1,-1,-1,alpha+beta-1)
        else:
            return (-1,1,alpha+beta-1,-1)
        
    else:
        
        return (1,1,-1,-1)

In [20]:
#test simplify

a = 1
b = 0

total_a = 0
total_b = 0

n=1000000

for i in range(n):
    (x,y,new_a,new_b) = simplify(a,b)
    
    if x==-1:
        total_a += new_a
        total_b += y
    elif y==-1:
        total_a += x
        total_b += new_b
    else:
        total_a += x
        total_b += y

print(str(total_a/n) + " " + str(a))
print(str(total_b/n) + " " + str(b))

1.0 1
0.0 0


In [21]:
def D(p):
    t = len(p)
    sample = np.full(t,-1)
    prob = np.array(p,copy=True)
    
    leaves = np.arange(t)
    np.random.shuffle(leaves)
    
    #each iteration represents a level in the tree
    while t > 1:
        
        new_leaves = []
        
        for i in range(0,t,2):            
            
            a = prob[leaves[i]]
            b = prob[leaves[i+1]]
            
            (x,y,new_a,new_b) = simplify(a,b)
            
            if x==-1:
                
                new_leaves.append(leaves[i])
                prob[leaves[i]] = new_a
                sample[leaves[i+1]] = y
                
            elif y==-1:
                
                new_leaves.append(leaves[i+1])
                prob[leaves[i+1]] = new_b
                sample[leaves[i]] = x

            else:
                
                new_leaves.append(leaves[i])
                prob[leaves[i]] = x
                sample[leaves[i+1]] = y
    
        if t%2 == 1:

            new_leaves.append(leaves[-1])
            
        t = len(new_leaves)
        leaves = np.array(new_leaves)
        np.random.shuffle(leaves)

    if t==1:
        
        p = random.uniform(0,1)
        
        if p < prob[leaves[0]]:
            sample[leaves[0]] = 1
        else:
            sample[leaves[0]] = 0
       
    return sample

In [22]:
p = [.5,.5,.25,.75]

n=10000
total = np.zeros(4)

for i in range(n):
    a = D(p)
    if np.sum(a) != np.sum(p):
        print("oof")
    #print(p)
    total += a
    
total/=n
print(total)

[0.5029 0.4982 0.2476 0.7513]


In [23]:
np.sum(p)

2.0

In [24]:
def D_prime(p):
    l = np.sum(p)
    
    p_prime = np.append(p,[math.ceil(l)-l])
    
    return D(p_prime)[:len(p)]

In [25]:

p = [.5,.4,.25,.75]

n=5000
total = np.zeros(4)

for i in range(n):
    a = D(p)
    if (np.sum(a) != math.floor(np.sum(p))) & (np.sum(a) != math.ceil(np.sum(p))):
        print("oof")
    #print(p)
    total += a
    
total/=n
print(total)

[0.5084 0.3924 0.2478 0.7562]


In [25]:
for i in range(0,1,2):
    print(i)

0
